In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from dotenv import load_dotenv
import os

load_dotenv('')

MODEL = "google/gemma-3-1b-it"
HF_TOKEN = os.getenv("HF_TOKEN") 

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL, token=HF_TOKEN)
model = AutoModel.from_pretrained(MODEL, token=HF_TOKEN)
model.to(device)

# 2. Function to get embeddings
def get_embedding(text):
    # tokenize
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(model.device)

    # forward pass
    with torch.no_grad():
        outputs = model(**inputs)
        hidden_states = outputs.last_hidden_state  # (batch, seq_len, hidden_dim)

    # mean pooling (ignores padding)
    mask = inputs["attention_mask"].unsqueeze(-1).expand(hidden_states.size())
    summed = torch.sum(hidden_states * mask, dim=1)
    counts = mask.sum(dim=1)
    mean_pooled = summed / counts

    # normalize (useful for cosine similarity)
    embedding = torch.nn.functional.normalize(mean_pooled, p=2, dim=1)

    return embedding[0].cpu().numpy()  # return as NumPy array

# 3. Example usage
text = "Gemma 3 is great for creating embeddings."
embedding = get_embedding(text)

print("Embedding vector length:", len(embedding))
print("First 10 values:", embedding[:10])

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from dotenv import load_dotenv
import os

load_dotenv('')

MODEL = "google/gemma-3-1b-it"
HF_TOKEN = os.getenv("HF_TOKEN") 

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(MODEL, token=HF_TOKEN)
model.to(device)

# Prompt
prompt = "Explain the difference between supervised and unsupervised learning in simple terms."

# Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        top_p=0.9,
        temperature=0.7
    )

# Decode
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Model output:\n", response)